In [1]:
!pip install openai
!pip install tiktoken
!pip install  cohere


import os, sys, cohere, tiktoken
from openai import OpenAI
from google.colab import userdata


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.8 MB/s eta 0:00:00


In [2]:

!pip install duckduckgo-search -q # Add the search library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.8 MB/s eta 0:00:00


In [3]:
import openai
print(f"OpenAI Library Version: {openai.__version__}")

OpenAI Library Version: 1.76.0


In [4]:
from google.colab import userdata # For API Key
import requests # Keep for potential future use, though not directly for DDG search
import json
import time # Needed for polling delay
import random # Needed for random delays (optional but good)
from duckduckgo_search import DDGS # Import the search library

In [5]:
from google.colab import drive
import requests

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:

img_link = "https://i.imgur.com/NtdR9Fw.png"  # Direct download link
img_link2 ="https://i.imgur.com/5QS5NvT.png"
img_link3 ="https://i.imgur.com/ljAahlI.jpeg"
img_link4 ="https://i.imgur.com/Rx27EoB.jpeg"
img_link5="https://i.imgur.com/lD64vvI.jpeg"
img_link6="https://i.imgur.com/CMFJfaR.jpeg"
img_link7="https://i.imgur.com/Xc4zUiw.jpeg"
img_link8="https://i.imgur.com/PHyvoLt.jpeg"
img_link9="https://i.imgur.com/R0RNcqz.jpeg"

In [9]:
client = OpenAI(api_key = userdata.get('API_KEY'))

In [10]:

# --- Define the exact fields you want consistently ---
requested_fields = [
    "producer_brand",
    "wine_name", # Using the more descriptive name
    "region", # Using the more descriptive name
    "grape_type", # Often useful if visible
    "alcohol_percent",     # Using standard name
    "price",
    "tasting_notes",
    "score",
]
fields_string = ", ".join(f'"{field}"' for field in requested_fields)

# --- Construct a more robust prompt ---
prompt_text = (
    f"Analyze the attached image to identify all visible wine bottles. "
    f"For each distinct bottle identified, extract its information. "
    f"Return the result ONLY as a JSON formatted list (an array), where each element in the list "
    f"is a JSON object representing one wine bottle. "
    f"Each JSON object should contain the following keys: {fields_string}. "
    f"If a specific piece of information for a key isn't clearly visible or readable on a label, "
    f"use the JSON value `null` for that key. Do not include any text before or after the JSON list."
)

In [11]:

try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img_link6,
                            # Optional: detail level ('low', 'high', 'auto') - 'high' might help with small text
                            # "detail": "high"
                        },
                    },
                ],
            }
        ],
        # Increased max_tokens, or remove if confident output is not excessively long
        max_tokens=1024,
    )

    raw_response_content = response.choices[0].message.content
    print("--- Raw Model Response ---")
    print(raw_response_content)
except Exception as e:
    print(f"An error occurred: {e}")

--- Raw Model Response ---
```json
[
    {
        "producer_brand": "DADA",
        "wine_name": "Art Wine #2",
        "region": null,
        "grape_type": null,
        "alcohol_percent": null,
        "price": "11.00",
        "tasting_notes": null,
        "score": null
    },
    {
        "producer_brand": "DADA",
        "wine_name": "Art Wine #3",
        "region": null,
        "grape_type": null,
        "alcohol_percent": null,
        "price": "11.00",
        "tasting_notes": null,
        "score": null
    },
    {
        "producer_brand": "DADA",
        "wine_name": "Art Wine",
        "region": null,
        "grape_type": null,
        "alcohol_percent": null,
        "price": null,
        "tasting_notes": null,
        "score": null
    },
    {
        "producer_brand": "DADA",
        "wine_name": "Art Wine #8",
        "region": null,
        "grape_type": null,
        "alcohol_percent": null,
        "price": null,
        "tasting_notes": null,
        "scor

In [12]:


# Encode the JSON response using the 'cl100k_base' encoding (used by GPT models)
encoding = tiktoken.encoding_for_model("gpt-4o")
num_tokens = len(encoding.encode(response.choices[0].message.content))

print(f"The JSON response has {num_tokens} tokens.")

The JSON response has 807 tokens.


In [13]:

from openai import OpenAI

In [14]:
if raw_response_content:
        try:
            if raw_response_content.startswith("```json"):
                clean_json_string = raw_response_content.strip().removeprefix("```json").removesuffix("```").strip()
            else:
                clean_json_string = raw_response_content.strip()

            parsed_wines = json.loads(clean_json_string)

            if isinstance(parsed_wines, list):
                print("\n--- Parsed Wine Data from Vision (List of Dicts) ---")
                # print(json.dumps(parsed_wines, indent=2)) # Optional: print full list
                print(f"Successfully parsed {len(parsed_wines)} wines from vision.")
            else:
                print("\n--- Parsed Data (Not a List as Expected) ---")
                print(json.dumps(parsed_wines, indent=2))
                print("Warning: The model did not return a JSON list as requested.")
                if isinstance(parsed_wines, dict): # Handle single object case
                     parsed_wines = [parsed_wines]
                     print("Stored the single object as a list with one element.")
                else:
                     parsed_wines = [] # Treat as empty if not list or dict

        except json.JSONDecodeError as e:
            print(f"\nError: Failed to decode JSON from the vision response.")
            print(f"JSONDecodeError: {e}")
            parsed_wines = []
        except Exception as e:
            print(f"\nAn unexpected error occurred during vision response parsing: {e}")
            parsed_wines = []
else:
    print("\nNo content received from Vision API call.")
    parsed_wines = []


--- Parsed Wine Data from Vision (List of Dicts) ---
Successfully parsed 12 wines from vision.


In [15]:
try:
    print(f"Uploading RAG file")
    uploaded_file = client.files.create(
        file=open('/content/drive/MyDrive/tescoRAG.json', "rb"), #file=open('/content/tescoRAG.json', "rb")  if using RAgfile uploaded on colab
        purpose='assistants' # IMPORTANT: Purpose must be 'assistants' for File Search/Assistants API
    )
    rag_file_id = uploaded_file.id
    print(f"File uploaded successfully. File ID: {rag_file_id}")

except FileNotFoundError:
    print(f"Error: RAG file not found at {'/content/tescoRAG.json'}")
    rag_file_id = None
except Exception as e:
    print(f"Error uploading file: {e}")
    rag_file_id = None

# Proceed only if file upload was successful
if not rag_file_id:
    print("Cannot proceed without a valid RAG File ID.")
    # Exit or handle error

Uploading RAG file
File uploaded successfully. File ID: file-AXF65yPgXuC6LmEvBLQ1LY


In [16]:
prompt_text2 = (
        f"You are Emy, a french witty lady sommelier expert of wines based in Ireland. Your goal is to help users find the perfect wine. "
        f"First, use the File Search tool to consult your attached wine catalog (Vector Store) to enrich any wine list provided by the user with details like typical price, ABV, grape varieties, and tasting notes found *only* in the provided files. Announce when you find a match in the RAG file. "
        f"If, after checking the file, key information like 'tasting_notes' and 'score' (rating) is still missing for a wine, THEN use the 'get_wine_reviews_from_web' function to search the web for that specific wine's reviews or ratings. "
        f"When you receive results from the function (which are search snippets), carefully read them and try to extract ratings (like points or stars) and key tasting descriptors. Summarize these findings concisely. "
        f"Finally, based on all gathered information and user preferences (like situation, budget), provide a recommendation and a sommelier-like description for the most suitable wine(s)."
        f"Always present factual information based on your tools. Be friendly, knowledgeable, and slightly charming. Return final structured wine data when appropriate."
    )

In [17]:
vector_store_id = None
if rag_file_id:
    # --- Step 2: Create Vector Store ---
    try:
        print("Creating a new Vector Store...")
        # WARNING: Using non-standard client.vector_stores path.
        vector_store = client.vector_stores.create(
            name="Emy Wine Catalog Store", # Give it a descriptive name
            file_ids=[rag_file_id]
        )
        vector_store_id = vector_store.id
        print(f"Vector Store created with ID: {vector_store_id}")
        # File processing might take a short time in the background
    except AttributeError as ae:
        print(f"\n--- AttributeError STILL Occurred ---")
        print(f"Error: {ae}")
        print(f"Type of client: {type(client)}")
        print(f"Has 'beta': {'beta' in dir(client)}")
        if 'beta' in dir(client):
            print(f"Has 'vector_stores': {'vector_stores' in dir(client.beta)}")
        print("------------------------------------")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Creating a new Vector Store...
Vector Store created with ID: vs_68180003b13081919ec1e616672029b4


In [18]:
if client and rag_file_id: # Proceed only if client and file ID are valid
    print("defining tools list")
    tools_list = [
        {"type": "file_search"}, # Keep File Search
        {
            "type": "function",
            "function": {
                "name": "get_wine_reviews_from_web",
                "description": "Searches the web (using DuckDuckGo) for reviews, ratings, or tasting notes for a specific wine based on its name and vintage (if available). Returns summaries of the top few search results.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "wine_query": {
                            "type": "string",
                            "description": "The full search query for the wine, including producer, name, and vintage if known (e.g., 'Yellow Tail Shiraz NV review', 'Campo Viejo Rioja Reserva 2018 tasting notes').",
                        }
                    },
                    "required": ["wine_query"],
                },
            }
        }
    ]

defining tools list


In [19]:
def get_wine_reviews_from_web_impl(wine_query):
        """ Performs targeted web searches for wine reviews/ratings and returns formatted snippets. """
        print(f"--- Function Call: Performing targeted web search for: {wine_query} ---")
        search_targets = [
            f"{wine_query} Vivino rating review", f"{wine_query} tasting notes",
            f"{wine_query} review",
        ]
        all_snippets = []
        urls_seen = set()
        max_total_snippets = 4

        try:
            with DDGS() as ddgs:
                for target_query in search_targets:
                    if len(all_snippets) >= max_total_snippets: break
                    print(f"  - Searching for: '{target_query}'")
                    try:
                        results = ddgs.text(target_query, max_results=2)
                        if not results: continue
                        for result in results:
                            if len(all_snippets) >= max_total_snippets: break
                            url = result.get('href')
                            if url and url not in urls_seen and result.get('body'):
                                snippet_data = {
                                    "query_used": target_query, "title": result.get('title'),
                                    "url": url, "snippet": result.get('body', '')[:350]
                                }
                                all_snippets.append(snippet_data)
                                urls_seen.add(url)
                                print(f"    Added snippet from: {url}")
                    except Exception as search_err: print(f"    Error during search for '{target_query}': {search_err}")
                    time.sleep(0.5) # Small delay between searches

            if not all_snippets:
                print("--- Function Call: No relevant web search snippets found overall. ---")
                return json.dumps({"results": "No relevant web search results found."})

            print(f"--- Function Call: Returning {len(all_snippets)} snippets. ---")
            return json.dumps({"results": all_snippets})
        except Exception as e:
            print(f"!!! Error during web search function execution: {e}")
            return json.dumps({"error": "Failed to perform web search.", "details": str(e)})


In [20]:
available_functions = {"get_wine_reviews_from_web": get_wine_reviews_from_web_impl}

In [21]:

assistant_id = None
try:
    assistant=client.beta.assistants.create(
        name="Emy Wine Assistant",
        instructions=prompt_text2,
        model="gpt-4o-mini",
        tools=tools_list,
        tool_resources={
                "file_search": {
                    "vector_store_ids": [vector_store_id]
                }
            }
    )
    assistant_id = assistant.id
    print(f"Assistant created/updated successfully! Assistant ID: {assistant_id}")
except Exception as e:
    print(f"An error occurred: {e}")

Assistant created/updated successfully! Assistant ID: asst_KAsB2ajZJKHdchOxv3hRIppq


In [22]:
thread=None
thread_id=None
thread = client.beta.threads.create()

In [23]:
if client and assistant_id and parsed_wines:
    print(f"client : {client}")
    print(f"assistant_id : {assistant_id}")
    print(f"parsed_wines : {parsed_wines}")

client : <openai.OpenAI object at 0x7921affc3510>
assistant_id : asst_KAsB2ajZJKHdchOxv3hRIppq
parsed_wines : [{'producer_brand': 'DADA', 'wine_name': 'Art Wine #2', 'region': None, 'grape_type': None, 'alcohol_percent': None, 'price': '11.00', 'tasting_notes': None, 'score': None}, {'producer_brand': 'DADA', 'wine_name': 'Art Wine #3', 'region': None, 'grape_type': None, 'alcohol_percent': None, 'price': '11.00', 'tasting_notes': None, 'score': None}, {'producer_brand': 'DADA', 'wine_name': 'Art Wine', 'region': None, 'grape_type': None, 'alcohol_percent': None, 'price': None, 'tasting_notes': None, 'score': None}, {'producer_brand': 'DADA', 'wine_name': 'Art Wine #8', 'region': None, 'grape_type': None, 'alcohol_percent': None, 'price': None, 'tasting_notes': None, 'score': None}, {'producer_brand': 'Finca Labarca', 'wine_name': None, 'region': 'Rioja', 'grape_type': None, 'alcohol_percent': None, 'price': '12.00', 'tasting_notes': None, 'score': None}, {'producer_brand': 'Finca Laba

In [24]:
#user preferences
# 1. Price Categories
price_categories = {
    "1": "Cheapest (<€10)",
    "2": "Mid-Range (€10-€20)",
    "3": "Fancy (>€20)",
    "4": "Any Budget"
}

# 2. Color Categories
color_categories = {
    "1": "Red",
    "2": "White",
    "3": "Rosé",
    "4": "Sparkling",
    "5": "Any Color/Type"
}

# 3. Occasion Categories
occasion_categories = {
    "1": "Casual Chilling",
    "2": "Fancy Party",
    "3": "Date Night",
    "4": "Failed Exam",
    "5": "Any Occasion"
}

# 4. Body Categories
body_categories = {
    "1": "Light-bodied (Delicate, easy to drink)",
    "2": "Medium-bodied (Balanced, versatile)",
    "3": "Full-bodied (Rich, bold, powerful)",
    "4": "Any Body Style"
}

# 5. Sweetness/Dryness Categories
dryness_categories = {
    "1": "Dry",
    "2": "Off-Dry",
    "3": "Sweet",
    "4": "Any Sweetness Level"
}

# 6. Flavour Profile Categories (Note: These are subjective & wines can fit multiple)
flavour_categories = {
    "1": "Fruity",
    "2": "Earthy / Mineral",
    "3": "Spicy / Oaked",
    "4": "Zesty / Crisp",
    "5": "Complex",
    "6": "Any Flavour Profile"
}
#user will chose input preferences
#my preferences
price ="2"
color="1"
occasion="2"
body="3"
dryness="1"
flavour="5"
food_pairing="Cheese Platter"
region="Spain"
dislikes="Nothing too sweet"

In [25]:
# Convert your Python list to a JSON string
wines_json_string = json.dumps(parsed_wines, indent=2) # Use indent
message_content_for_assistant= (
    f" Okay Emy, I have processed the image/shelf data. Here is the list of wines identified: {wines_json_string}, Please perform the following steps:"
    f"Use your File Search tool to consult the attached wine catalog (RAG file)."
    f"For each wine in the list above, try to enrich it by adding or correcting: alcohol_percent, region (make more specific), grape_type, tasting_notes,  and add a typical price_eur field based only on the catalog file. Announce when you find a match."
    f"If tasting_notes and score or price are still missing after checking the file, use the get_wine_reviews_from_web function to find that information online for each wine lacking it."
    f"Once all enrichment is done (from file and potentially web search), analyze the completed list."
    f"Return the fully enriched list in JSON format."
    f"These are the user preferences: Price: {price_categories[price]}, Color: {color_categories[color]}, Occasion: {occasion_categories[occasion]}, Body: {body_categories[body]}, Dryness: {dryness_categories[dryness]}, Flavour: {flavour_categories[flavour]}, Food Pairing: {food_pairing}, region: {region}, dislikes{dislikes}"
    f"Finally, based on these user preferences and the fully enriched list of wines, select the single best recommendation wine to provide a sommelier description for it."
    f"Now, please act as the friendly, witty, knowledgeable French sommelier Emy. Provide a concise and engaging recommendation description ONLY for this specific selected wine. Explain clearly why it's a good fit for my stated preferences and situation. Your response should be only the final recommendation text."
)

In [26]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= message_content_for_assistant
)

In [27]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
print(f"Run created: {run.id}, Initial Status: {run.status}")

Run created: run_rAWVkYXKT0UBtBuXkBCOHmY1, Initial Status: queued


In [28]:
latestRun=client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(latestRun.status)

queued


In [29]:
while run.status in ['queued', 'in_progress', 'cancelling']:
        print(f"Polling Run Status: {run.status}...")
        time.sleep(4) # Wait a bit longer
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id) # Use standard path

        if run.status == 'requires_action':
            print("Run requires action: Function call detected.")
            required_action = run.required_action
            if required_action and required_action.type == 'submit_tool_outputs':
                tool_outputs = []
                for tool_call in required_action.submit_tool_outputs.tool_calls:
                    if tool_call.type == "function":
                        function_name = tool_call.function.name
                        try:
                             function_args = json.loads(tool_call.function.arguments)
                        except json.JSONDecodeError:
                             print(f"!!! Error: Could not parse function arguments: {tool_call.function.arguments}")
                             function_args = {} # Proceed with empty args? Or submit error?

                        print(f"  - Assistant wants to call: {function_name}({function_args})")

                        if function_name in available_functions:
                            function_to_call = available_functions[function_name]
                            try:
                                function_response = function_to_call(**function_args)
                                print(f"  - Function executed. Response snippet: {function_response[:200]}...")
                                tool_outputs.append({
                                    "tool_call_id": tool_call.id,
                                    "output": function_response,
                                })
                            except Exception as func_e:
                                print(f"!!! Error executing function {function_name}: {func_e}")
                                tool_outputs.append({
                                     "tool_call_id": tool_call.id,
                                     "output": json.dumps({"error": f"Failed execution: {func_e}"})
                                })
                        else:
                            print(f"!!! Warning: Unknown function requested: {function_name}")
                            tool_outputs.append({
                                "tool_call_id": tool_call.id,
                                "output": json.dumps({"error": f"Function '{function_name}' unavailable."})
                             })
                # Submit outputs if any
                if tool_outputs:
                    try:
                        print("Submitting tool outputs...")
                        run = client.beta.threads.runs.submit_tool_outputs( # Use standard path
                            thread_id=thread.id,
                            run_id=run.id,
                            tool_outputs=tool_outputs
                        )
                        print("Tool outputs submitted.")
                    except Exception as submit_e:
                        print(f"!!! Error submitting tool outputs: {submit_e}")
                        break # Exit loop on submission error
            else:
                 print("Requires action, but not submit_tool_outputs type?")
                 break # Exit loop if action type is unexpected

        elif run.status == 'completed':
            print("Run completed successfully.")
            break

        elif run.status in ['failed', 'cancelled', 'expired']:
            print(f"Run finished with unsuccessful status: {run.status}")
            last_error = getattr(run, 'last_error', None) # Safely get last_error
            if last_error: print(f"  Error: {getattr(last_error, 'message', 'No error message provided')}")
            break

Polling Run Status: queued...
Polling Run Status: in_progress...
Polling Run Status: in_progress...
Run requires action: Function call detected.
  - Assistant wants to call: get_wine_reviews_from_web({'wine_query': 'DADA Art Wine #2 review'})
--- Function Call: Performing targeted web search for: DADA Art Wine #2 review ---
  - Searching for: 'DADA Art Wine #2 review Vivino rating review'
    Added snippet from: https://www.vivino.com/US/en/bodega-finca-las-moras-dada-art-2/w/1398258
    Added snippet from: https://www.reddit.com/r/wine/comments/ride6m/how_useful_do_you_find_vivino_ratings/
  - Searching for: 'DADA Art Wine #2 review tasting notes'
    Added snippet from: https://www.wine-searcher.com/find/finca+moras+dada+art+two+st+juan+argentina
  - Searching for: 'DADA Art Wine #2 review review'
    Added snippet from: https://www.nataliemaclean.com/wine-reviews/dada-de-finca-las-moras-2012/188607
--- Function Call: Returning 4 snippets. ---
  - Function executed. Response snippet:

In [30]:
if run.status == 'completed':
        print("\nRetrieving final messages...")
        messages_page = client.beta.threads.messages.list( # Use standard path
            thread_id=thread.id,
            order='asc'
        )
        if messages_page.data:
            print("\n--- Final Thread Messages ---")
            for message in messages_page.data:
                role = message.role
                content_text = ""
                if message.content and isinstance(message.content, list):
                     for content_item in message.content:
                          if content_item.type == 'text':
                               content_text += content_item.text.value + "\n"
                          elif content_item.type == 'image_file':
                               content_text += f"[Image File: {content_item.image_file.file_id}]\n"
                print(f"[{role.capitalize()}]:\n{content_text.strip()}")
                print("-" * 20)
        else:
            print("No messages found after run completion.")
else:
    print(f"\nRun did not complete successfully (Final Status: {run.status}). Cannot show final response.")



Retrieving final messages...

--- Final Thread Messages ---
[User]:
Okay Emy, I have processed the image/shelf data. Here is the list of wines identified: [
  {
    "producer_brand": "DADA",
    "wine_name": "Art Wine #2",
    "region": null,
    "grape_type": null,
    "alcohol_percent": null,
    "price": "11.00",
    "tasting_notes": null,
    "score": null
  },
  {
    "producer_brand": "DADA",
    "wine_name": "Art Wine #3",
    "region": null,
    "grape_type": null,
    "alcohol_percent": null,
    "price": "11.00",
    "tasting_notes": null,
    "score": null
  },
  {
    "producer_brand": "DADA",
    "wine_name": "Art Wine",
    "region": null,
    "grape_type": null,
    "alcohol_percent": null,
    "price": null,
    "tasting_notes": null,
    "score": null
  },
  {
    "producer_brand": "DADA",
    "wine_name": "Art Wine #8",
    "region": null,
    "grape_type": null,
    "alcohol_percent": null,
    "price": null,
    "tasting_notes": null,
    "score": null
  },
  {
   

#detach the file when not coding

In [31]:

# --- Replace with your actual IDs ---
vector_store_id_to_delete = vector_store_id # The ID you got from client.beta.vector_stores.create
# -----------------------------------

if vector_store_id_to_delete:
    try:
        print(f"Attempting to delete Vector Store: {vector_store_id_to_delete}...")
        response = client.vector_stores.delete(vector_store_id_to_delete)
        print(f"Deletion response: {response}")
        if response.deleted:
             print(f"Vector Store {vector_store_id_to_delete} deleted successfully.")
             print("Associated storage costs should now stop.")
        else:
             print(f"Failed to delete Vector Store {vector_store_id_to_delete}.")

    except Exception as e:
        print(f"Error deleting Vector Store {vector_store_id_to_delete}: {e}")
else:
    print("No Vector Store ID provided to delete.")

# Note: Your Assistant still exists but its file_search tool no longer points
# to this deleted Vector Store. You'd need to update the Assistant with a new
# Vector Store ID if you want to use RAG again later.

Attempting to delete Vector Store: vs_68180003b13081919ec1e616672029b4...
Deletion response: VectorStoreDeleted(id='vs_68180003b13081919ec1e616672029b4', deleted=True, object='vector_store.deleted')
Vector Store vs_68180003b13081919ec1e616672029b4 deleted successfully.
Associated storage costs should now stop.


In [32]:
rag_file_path_colab= '/content/tescoRAG.json'
# --- Step 2: Delete the RAG file from Colab local storage ---
print(f"\nAttempting to delete local Colab file: {rag_file_path_colab}...")
if os.path.exists(rag_file_path_colab):
    try:
        os.remove(rag_file_path_colab)
        print(f"Successfully deleted local file: {rag_file_path_colab}")
    except OSError as e:
        print(f"Error deleting local file {rag_file_path_colab}: {e}")
    except Exception as e:
         print(f"An unexpected error occurred deleting local file: {e}")
else:
    print(f"Local file {rag_file_path_colab} not found. Skipping deletion.")

print("\nCleanup process finished.")


Attempting to delete local Colab file: /content/tescoRAG.json...
Local file /content/tescoRAG.json not found. Skipping deletion.

Cleanup process finished.
